# Quant Finance - Clase 1

## Web Scraping de la pagina de RAVA

Modulos necesarios

Codigo.data_byma (en github) ;

Codigo.utils_opciones_byma (en github)

bs4,

requests,

pandas,

matplotlib,

datetime,

calendar,

Instalarlos con Anaconda, comando Conda o comando pip install

### Importo los modulos

In [5]:
import sys
sys.path.append('..')
#Para poder leer los modulos dentro del directorio 
from Codigo.data_byma import *
from Codigo.utils_opciones_byma import *

### Metodo para el web scraping

In [6]:
panel_rava = web_scraping_opciones_rava()
panel_rava.head(20)

,Especie,Último,% Día,Anterior,Apertura,Mínimo,Máximo,Hora,Vol. Nominal,Vol. Efectivo
0,AGRC12.0AG,1.990,165.33,0.750,2.000,1.990,2.00,15:17,29.0,5796.0
1,ALUC13.9AG,27.999,2.19,27.400,27.000,26.000,28.00,16:56,1343.0,3516153.0
2,ALUC13.9OC,29.500,0.00,29.500,27.850,27.400,29.50,16:54,1101.0,3094285.0
3,ALUC20.9AG,23.000,-4.17,24.000,23.000,23.000,23.00,14:57,4.0,9200.0
4,ALUC27.0AG,16.000,3.84,15.408,16.250,16.000,18.00,16:24,105.0,187971.0
5,ALUC31.0AG,11.000,25.86,8.740,13.500,11.000,13.50,15:07,17.0,21200.0
6,ALUC31.0OC,13.500,-12.07,15.353,12.500,12.000,13.50,16:50,88.0,110751.0
7,ALUC33.0AG,10.000,18.34,8.450,10.000,10.000,10.10,14:57,3.0,3010.0
8,ALUC35.0AG,9.400,10.60,8.499,9.000,8.500,9.70,16:15,104.0,94720.0
9,ALUC37.0AG,6.000,-7.61,6.494,6.494,5.200,6.50,16:58,194.0,109941.0


### Que hace internamente el metodo??

#### 1) Beautiful Soup y otros imports

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

#### 2) Como usamos BeautifulSoup y requests

In [ ]:
url = 'http://www.rava.com/precios/panel.php?m=OPC'
page = requests.get(url)
soup = BeautifulSoup(page.text, 'lxml')


In [ ]:
# En soup voy a encontrar el codigo html de la pagina
soup

In [ ]:
# En opciones tengo el contenido de la tabla correspondiente
opciones_html = soup.find('table', {'class': 'tablapanel2'})

opciones_html

In [ ]:
# Filas ya va ordenando cada una de las opciones
filas = opciones_html.find_all('tr')
filas

#### 3) Como extraemos la informacion

In [ ]:
tr = filas[0]
tr

In [ ]:
td = tr.find_all('td')
td


In [ ]:
row = [tr.text for tr in td]
row

In [ ]:
# inicializo una lista vacia, alli voy a ir poniendo el contenido de cada fila
l = []
l.append(row)

l

In [ ]:
#Ahora si, voy linea por linea a por el contenido de cada fila (dejo la primera linea afuera del loop para que se vea)

tr = filas[1]
tr

In [ ]:
td = tr.find_all('td')
row = [tr.text for tr in td]
row

In [ ]:
#Reemplazo ',' por '.' y paso el string a float
for index in [1,2,3,4,5,6,8,9]:
    row[index] = row[index].replace('.', '')
    row[index] = row[index].replace(',', '.')
    try:
        row[index] = float(row[index])
    except:
        #Si no encuentra valor pone -99.99
        row[index] = -99.99

l.append(row)
    
l
    
    

In [ ]:
#Ahora si, todas las filas
for tr in filas[2:]:
    td = tr.find_all('td')
    row = [tr.text for tr in td]

    #Reemplazo ',' por '.' y paso el string a float
    for index in [1,2,3,4,5,6,8,9]:
        row[index] = row[index].replace('.', '')
        row[index] = row[index].replace(',', '.')
        try:
            row[index] = float(row[index])
        except:
            #Si no encuentra valor pone -99.99
            row[index] = -99.99

    l.append(row)

In [ ]:
# Como se ve l despues del ciclo?
l

#### 4) Lo convierto en un DataFrame

In [ ]:
panel_rava = pd.DataFrame(l[1:], columns=l[0])

panel_rava

## Panel acciones (bonus)

In [ ]:
panel_acciones = obtener_panel_acciones()
panel_acciones

## Del panel de Rava crudo a otro con informacion customizada

### Metodo para generar el panel customizado

In [ ]:
panel_opciones_byma_all_raw = obtener_panel_opciones_byma(ticker = '', clean_flag = False)
panel_opciones_byma_all_raw

In [ ]:
panel_opciones_byma_all_clean = obtener_panel_opciones_byma(ticker = '', clean_flag = True)
panel_opciones_byma_all_clean

In [ ]:
panel_opciones_byma_GGAL_clean = obtener_panel_opciones_byma('GGAL', True)
panel_opciones_byma_GGAL_clean

### Que hace el metodo internamente?



#### 1) Levanta los paneles de opciones y de acciones y genera una copia del de opciones

In [ ]:
panel_rava = web_scraping_opciones_rava()
#Obtengo el panel crudo de acciones (para el spot)
panel_acciones = obtener_panel_acciones()

#Genero el nuevo panel
panel_opciones = panel_rava.copy()


#### 2) Inicializo las columnas nuevas a agregar del Dataframe

In [ ]:
#Le agrego las nuevas columnas (ojo con los tipos string, float, int, datetime)
panel_opciones['Ticker_Opcion'] = ''
panel_opciones['Ticker_Stock'] = ''
panel_opciones['Tipo'] = ''
panel_opciones['Strike'] = 0.0
panel_opciones['ExpiryMonthName'] = ''
panel_opciones['ExpiryMonthNumber'] = 0
panel_opciones['ExpiryDate'] = datetime.date.today()
panel_opciones['Spot'] = 0.0
panel_opciones['TTM'] = 0
panel_opciones['Moneyness'] = 0.0

panel_opciones

#### 3) Recorro fila por fila para llenar las nuevas columnas (1 ejemplo)

##### *) Desgloso la especie en ticker, strike, vencimiento y tipo

In [ ]:
fila = 0


panel_opciones.Ticker_Opcion.values[fila] = panel_opciones.Especie.values[fila][0:3]

panel_opciones.Tipo.values[fila] = panel_opciones.Especie.values[fila][3:4]
panel_opciones.Strike.values[fila] = panel_opciones.Especie.values[fila][4:8]
panel_opciones.ExpiryMonthName.values[fila] = panel_opciones.Especie.values[fila][8:10]

panel_opciones.head(2)

##### *) Uno a uno voy generando lo necesario

In [ ]:
panel_opciones.Ticker_Stock.values[fila] = conversor_ticker(panel_opciones.Ticker_Opcion.values[fila])

panel_opciones.Tipo.values[fila] = panel_opciones.Tipo.values[fila].replace('V', 'P')

panel_opciones.head(2)

In [ ]:
# Artilugio que lo tuve que usar NO RECUERDO POR QUE! :)

try:
    int(panel_opciones.ExpiryMonthName.values[fila][0])
    panel_opciones.Strike.values[fila] = float(panel_opciones.Strike.values[fila] + int(panel_opciones.ExpiryMonthName.values[fila][0]))
    panel_opciones.ExpiryMonthName.values[fila] = panel_opciones.ExpiryMonthName.values[fila][1]
except:
    panel_opciones.Strike.values[fila] = float(panel_opciones.Strike.values[fila])
    
    

In [ ]:
#conversor_ticker('AGR')

#panel_opciones.Especie.values[fila][0:3]

#panel_opciones.Ticker_Opcion.values[fila]\

#conversor_ticker(panel_opciones.Ticker_Opcion.values[fila])

#panel_opciones.Ticker_Stock.values[fila]



##### *) Obtengo el precio spot del ticker con un metodo interno y aprovechando el panel de acciones

In [ ]:
panel_opciones.Spot.values[fila] = obtener_spot_ticker(panel_acciones, panel_opciones.Ticker_Stock.values[fila])

##### *) Obtengo el Time to maturity (en dias)

In [ ]:
try:
    panel_opciones.ExpiryMonthNumber.values[fila] = mes_nombre_a_numero(panel_opciones.ExpiryMonthName.values[fila])
except:
    panel_opciones.ExpiryMonthNumber.values[fila] = 0

panel_opciones.ExpiryDate.values[fila] = fecha_expiracion(panel_opciones.ExpiryMonthNumber.values[fila])
hoy = datetime.date.today()
panel_opciones.TTM.values[fila] = (panel_opciones.ExpiryDate.values[fila] - hoy).days

panel_opciones

##### *) Por ultimo calculo el moneyness de valor informativo importante

In [ ]:
panel_opciones.Moneyness.values[fila] = panel_opciones.Spot.values[fila] / panel_opciones.Strike.values[fila]

In [ ]:
panel_opciones.head(2)

#### 3bis) Recorro fila por fila para llenar las nuevas columnas


In [ ]:

for fila in range(1,len(panel_opciones.Especie.values)):
    panel_opciones.Ticker_Opcion.values[fila] = panel_opciones.Especie.values[fila][0:3]

    panel_opciones.Ticker_Stock.values[fila] = conversor_ticker(panel_opciones.Ticker_Opcion.values[fila])

    panel_opciones.Tipo.values[fila] = panel_opciones.Especie.values[fila][3:4]
    panel_opciones.Tipo.values[fila] = panel_opciones.Tipo.values[fila].replace('V', 'P')

    panel_opciones.Strike.values[fila] = panel_opciones.Especie.values[fila][4:8]

    panel_opciones.ExpiryMonthName.values[fila] = panel_opciones.Especie.values[fila][8:10]

    try:
        int(panel_opciones.ExpiryMonthName.values[fila][0])
        panel_opciones.Strike.values[fila] = float(panel_opciones.Strike.values[fila] + int(panel_opciones.ExpiryMonthName.values[fila][0]))
        panel_opciones.ExpiryMonthName.values[fila] = panel_opciones.ExpiryMonthName.values[fila][1]
    except:
        panel_opciones.Strike.values[fila] = float(panel_opciones.Strike.values[fila])

    try:
        panel_opciones.ExpiryMonthNumber.values[fila] = mes_nombre_a_numero(panel_opciones.ExpiryMonthName.values[fila])
    except:
        panel_opciones.ExpiryMonthNumber.values[fila] = 0

    panel_opciones.ExpiryDate.values[fila] = fecha_expiracion(panel_opciones.ExpiryMonthNumber.values[fila])


    panel_opciones.Spot.values[fila] = obtener_spot_ticker(panel_acciones, panel_opciones.Ticker_Stock.values[fila])


    hoy = datetime.date.today()
    panel_opciones.TTM.values[fila] = (panel_opciones.ExpiryDate.values[fila] - hoy).days

    panel_opciones.Moneyness.values[fila] = panel_opciones.Spot.values[fila] / panel_opciones.Strike.values[fila]



In [ ]:
#Asi queda entonces el panel
panel_opciones

#### 4) Filtro por ticker


In [ ]:
#Si se provee un ticker, se filtra por ticker, si no, se provee el panel completo

ticker = 'ALUA'

if ticker == '':
    pass
else:
    panel_opciones = panel_opciones[panel_opciones.Ticker_Stock == ticker]
    
panel_opciones

#### 5) Panel limpio

In [ ]:
panel_limpio = panel_opciones.copy()

clean_flag = True

# Panel limpio: cambio de nombre, reordenamiento y limpieza minima del panel
if clean_flag == True:
    # Sacar opciones sobre bonos
    panel_limpio = panel_limpio[~panel_limpio.Especie.str.contains('A24')]

    panel_limpio = panel_limpio[
            ['Especie', 'Ticker_Stock', 'Spot', 'Tipo', 'Strike', 'TTM', 'Último', 'Moneyness']]

    panel_limpio = panel_limpio.rename(
            columns={'Ticker_Stock': 'Ticker', 'Tipo': 'CallPut', 'Último': 'Last'}, inplace=False)

    panel_limpio = panel_limpio.sort_values(['TTM', 'Ticker', 'Strike', 'CallPut'])

    # Remover TTMs mayores de un año o negativos (por algun error)
    panel_limpio = panel_limpio[panel_limpio.TTM < 364]
    panel_limpio = panel_limpio[panel_limpio.TTM > 0]

    panel_limpio = panel_limpio.reset_index()
    del panel_limpio['index']
    
panel_limpio

## Ploteando los datos

### Para cada vencimiento, grafico los strikes

In [ ]:
# BYMA


panel_opciones_ggal_byma_limpio = obtener_panel_opciones_byma('GGAL', True)

TTMs = list(set(panel_opciones_ggal_byma_limpio.TTM.values))
TTMs

In [ ]:
# El proximo vencimiento (usualemtne mas liquido)
TTM = min(TTMs)

panel_opciones_ggal_byma_limpio[(panel_opciones_ggal_byma_limpio.TTM == TTM)].head(20)

In [ ]:
from matplotlib import pyplot as plt
CP = "C"
for TTM in TTMs:
    Ks = panel_opciones_ggal_byma_limpio[(panel_opciones_ggal_byma_limpio.TTM == TTM) & (panel_opciones_ggal_byma_limpio.CallPut == CP)].Strike.values
    precios = panel_opciones_ggal_byma_limpio[(panel_opciones_ggal_byma_limpio.TTM == TTM) & (panel_opciones_ggal_byma_limpio.CallPut == CP)].Last.values
    plt.plot(Ks, precios,'*', label='TTM = '+str(TTM))


plt.legend()

plt.xlabel('Strike')
plt.ylabel('Precio de mercado del Call')
plt.title('Precio de mercado de las opciones Call con respecto al Strike')
    
plt.show()
    

In [ ]:

CP = "P"
for TTM in TTMs:
    Ks = panel_opciones_ggal_byma_limpio[(panel_opciones_ggal_byma_limpio.TTM == TTM) & (panel_opciones_ggal_byma_limpio.CallPut == CP)].Strike.values
    precios = panel_opciones_ggal_byma_limpio[(panel_opciones_ggal_byma_limpio.TTM == TTM) & (panel_opciones_ggal_byma_limpio.CallPut == CP)].Last.values
    plt.plot(Ks, precios,'*', label='TTM = '+str(TTM))


plt.legend()

plt.xlabel('Strike')
plt.ylabel('Precio de mercado del Call')
plt.title('Precio de mercado de las opciones Call con respecto al Strike')
    
plt.show()

In [ ]:
Ks = list(set(panel_opciones_ggal_byma_limpio.Strike.values))

In [ ]:

CP = "C"
for K in Ks:
    TTMs = panel_opciones_ggal_byma_limpio[(panel_opciones_ggal_byma_limpio.Strike == K) & (panel_opciones_ggal_byma_limpio.CallPut == CP)].Strike.values
    precios = panel_opciones_ggal_byma_limpio[(panel_opciones_ggal_byma_limpio.Strike == K) & (panel_opciones_ggal_byma_limpio.CallPut == CP)].Last.values
    if len(TTMs)>1:
        plt.plot(TTMs, precios,'*', label='Strike = '+str(K))

plt.legend()

plt.xlabel('TTM')
plt.ylabel('Precio de mercado del Call')
plt.title('Precio de mercado de las opciones Call con respecto al TTM')
    
plt.show()
    